<a href="https://colab.research.google.com/github/jugernaut/MACTI-programacionparalelo/blob/erick/03_CUDA/02_Algoritmos_CUDA_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Algoritmos CUDA</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M.en.C. Miguel Angel Pérez León</i></h5>
    <h5 align="center"><i>Ayudante: Jesús Iván Coss Calderón</i></h5>
    <h5 align="center"><i>Ayudante: Mario Arturo Nieto Butron</i></h5>
  <h5 align="center"><i>Materia: Seminario de programación en paralelo</i></h5>
  </font>

# Introducción

Como ya se menciono previamente, *CUDA* hace uso de las *GPU's* (dispositivos de cómputo de propósito especifico), estos dispositivos están optimizados para trabajar con imágenes y resulta que una imagen dentro de una computadora se representa mediante elementos matemáticos como matrices o vectores.

De hecho la mayoría de los formatos de imágenes más comunes (*.jpg, .jpeg, .png*) consideran a la imagen como una matriz de pixeles o un mapa de bits.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Imagenes/CUDA/smile.png?raw=1" width="600">
</center>



# Suma de Vectores

El algoritmo más sencillo que podemos comenzar a analizar es la suma de vectores y aunque suene trivial, la realidad es que hasta antes de este momento toda suma de vectores que hayas realizado previamente **se ejecuto de manera secuencial**, lo cuál significa un desperdicio de recursos.

Gracias a *CUDA* (aunque también se puede realizar con *OpenMP* y *MPI*) esta operación elemental se puede realizar en paralelo y gracias a esto optimizar recursos, lo que se traduce en un menor tiempo de ejecución.

## ¿Cómo funciona?

La forma tradicional de como funciona la suma de vectores, se basa en la definición formal "se realiza entrada a entrada", sin embargo nunca nos dijeron que esta "suma entrada a entrada" se puede realizar en paralelo.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Imagenes/CUDA/sumavect.png?raw=1" width="600">
</center>


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-nq4w423i
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-nq4w423i
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

// Kernel (funcion) que se invoca desde el Host y se ejecuta en un dispositivo
__global__ void suma_vectores(int* c, const int* a, const int* b, int size) {
    // polinomio de direccionamiento
    // ¡¡OJO 2 bloques(0 y 1), dim = 3, thread por bloque de 0-2!!
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < size) {
        // descomnetar para debugear
        //printf("%d \n",blockIdx.x);
        //printf("%d \n",blockDim.x);
        //printf("%d \n",threadIdx.x);
        c[i] = a[i] + b[i];
    }
}

// Funcion auxiliar que encapsula la suma con CUDA
void suma_CUDA(int* c, const int* a, const int* b, int tam) {
    int* dev_a = nullptr;
    int* dev_b = nullptr;
    int* dev_c = nullptr;

    // Reservamos espacio de memoria para los datos, 2 de entrada y una salida
    cudaMalloc((void**)&dev_c, tam * sizeof(int));
    cudaMalloc((void**)&dev_a, tam * sizeof(int));
    cudaMalloc((void**)&dev_b, tam * sizeof(int));

    // Copiamos los datos de entrada desde el CPU a la memoria del GPU
    cudaMemcpy(dev_a, a, tam * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, tam * sizeof(int), cudaMemcpyHostToDevice);

    // Se invoca al kernel en el GPU con un hilo por cada elemento
    // 2 es el numero de bloques y (tam + 1)/2 es el numero de hilos en cada bloque
    suma_vectores<<<2, (tam + 1) / 2>>>(dev_c, dev_a, dev_b, tam);

    // Esta funcion espera a que termine de ejecutarse el kernel y
    // devuelve los errores que se hayan generado al ser invocado
    cudaDeviceSynchronize();

    // Copiamos el vector resultado de la memoria del GPU al CPU
    cudaMemcpy(c, dev_c, tam * sizeof(int), cudaMemcpyDeviceToHost);

    // Se libera la memoria empleada
    cudaFree(dev_c);
    cudaFree(dev_a);
    cudaFree(dev_b);
}

// Funcion principal que sirve de prueba para el algoritmo
int main(int argc, char** argv) {

    // Datos de entrada para nuestra funcion
    const int tam = 5;
    const int a[tam] = {  1,  2,  3,  4,  5 };
    const int b[tam] = { 10, 20, 30, 40, 50 };
    int c[tam] = { 0 };

    // Se llama a la funcion que encapsula el Kernel
    suma_CUDA(c, a, b, tam);

    // Mostramos resultado
    printf("{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {%d, %d, %d, %d, %d}\n", c[0], c[1], c[2], c[3], c[4]);

    // Se liberan recursos
    cudaDeviceReset();

    return 0;
}

{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {11, 22, 33, 44, 55}



## Ventajas

Debido a lo que ya conocemos respecto al funcinamiento y desempeño de *CUDA*, podemos afirmar que este tipo de operaciones (suma de vectores) se realizan de manera más sencilla en un *GPU*.

Sean $\vec{a}=\{1,2,3,4,5\}$ y $\vec{b}=\{10,20,30,40,50\}$ entonces $\left(a_{0},b_{0}\right)$ se envían al núcleo 0, $\left(a_{1},b_{1}\right)$ se envían al núcleo 1, así sucesivamente hasta $\left(a_{n-1},b_{n-1}\right)$ se envían al núcleo $n-1$. En caso de que existan más entradas que nucleos, entonces se tendría que esperar a liberar alguno de los núcleos previamente empleados, sin embargo debido a la arquitectura de los *GPU's* sabemos que cada *GPU* posee muchos nucleos, **en las tarjetas más recientes podemos hablar del orden de miles**.

De igual manera como se midió el tiempo de ejecución con *OpenMP* o *MPI*, existe forma de medir el tiempo de ejecución de los algoritmos usando *CUDA*.

¿Cuáles son los ordenedes de complejidad a los que pertenecen ambas versiones de la suma de vectores, secuencial y en paralelo?.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/Cuda/gpucpu.jpg?raw=1" width="600">
</center>


## Desventajas

La desventaja más notoria en el algoritmo anterior, es el **cuello de botella** que se genera tanto al enviar los datos a procesar al *GPU*, como al extraerlos del *GPU*, sin embargo es claro que a mayor cantidad de datos a procesar también se tendría una mayor ganancia en tiempo, lo que se traduce en un menor tiempo de ejecución.

Este tipo de características (gran cantidad de datos a procesar) normalmente se encuentran en muchas áreas de las ciencias e ingenierías, por ejemplo ***machine learning***, por mencionar alguna.

# Operaciones con matrices

Una vez que ya se comprendió el desempeño de *CUDA* para la suma de vectores, es sencillo extender su aplicación a operaciones con matrices, por ejemplo la **suma de matrices**.

Por lo general cuando se hace uso de modelos matemáticos, estos toman la forma de matriz, uno de los más conocidos son las **redes neuronales**. Estas redes neuronales toman forma de matriz y en cada entrada de la matriz se tiene una neurona, y a su vez cada neurona puede ser representada por un escalar o un vector o incluso otra matriz.

Es por este motivo que las *GPU's* y en particular *CUDA* ha mostrado un excelente desempeño en el proceso de entrenamiento de las redes neuronales, más recientemente han surgido nuevos dispositivos de cómputo especifico como los *TPU's* (unidades de procesamiento tensorial), dispositivos que desde que fueron diseñados se contemplo su uso para el área de inteligencia artificial.

Volviendo al funcionamiento de *CUDA*, veamos una parte fundamental del mismo, la **jerarquía de memoria** de *CUDA* se estructura de la siguiente manera.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/Cuda/memtotal.png?raw=1" width="600">
</center>

## Entendiendo CUDA

Queda como ejercicio, realizar la suma de matrices empleando *CUDA*.



## Hint (polinomio de direccionamiento)

Imaginemos que queremos "aplanar" una matríz para representarla con un vector, es decir que necesitamos almacenar (y recuperar) cada una de las entradas de una matriz en un vector, ¿cómo le hacemos?.

La respuesta es mediante el **polinomio de direccionamiento**, este polinomio nos indica mediante una sola entrada, la localidad del vector que le corresponde a cada uno de los elementos de una matriz.

Supongamos que queremos almacenar los elementos de la matriz $A$ en una lista o vector.

Sea

$$A\in M_{2x2}=\left(\begin{array}{cc}
3_{(0,0)} & 6_{(0,1)}\\
7_{(1,0)} & 9_{(1,1)}
\end{array}\right)$$

La idea sería que estos elementos se alamcenen de la siguiente forma.

$$A=\left[\begin{array}{cccc}
3 & 6 & 7 & 9\end{array}\right]$$

Nos gustaría que la entrada $(0,0)$ de $A$ fuera mapeada a la localidad 0 de la lista y así sucesivamente hasta llegar a que la entrada $(1,1)$ se mapeara a la localidad 3 del arreglo, es decir

\begin{array}{cc}
f((0,0))=0 & f((0,1))=1\\
f((1,0))=2 & f((1,1))=3
\end{array}

Podríamos pensar que una buena forma de definir a $f$, seria $f((x,y))=x+y$, pero veamos que sucede al probarla.

\begin{array}{c}
f((0,0))=0+0=0.......\text{¡bien!}\\
f((0,1))=0+1=1.......\text{¡bien!}
\end{array}

Vamos bien, veamos que sucede con los elementos restantes.

\begin{array}{c}
f((1,0))=1+0=1.......\text{¡colisión!}\\
f((0,1))=1=f((1,0))
\end{array}

Dado que se tuvo una colisión, es necesario re-definirla de otra manera menos ingenua. Veamos que sucede si definimos a $f$ de la siguiente manera.

$$f((x,y))=2x+y$$

Al probarla, lo que obtenemos es.

\begin{array}{c}
f((0,0))=2*0+0=0\\
f((0,1))=2*0+1=1\\
f((1,0))=2*1+0=2\\
f((1,1))=2*1+1=3
\end{array}

Esta función no muestra colisiones (al menos en el dominio y codominio definidos), incluso se podría probar que no presentará colisiones para ningún par de tuplas de naturales.

Así que podemos pensar, que para el caso particular de matrices bidimensionales $A_{(i,j)}\in M_{ren \times col}$ podemos definir la función hash (polinomio de direccionamiento) que mapea localidades de dicha matriz en una lista (arreglo) unidimensional de la siguiente forma.

$$f((i,j))=col*i+j$$

¿Podemos extender este polinomio a objetos de 3 dimensiones, largo, ancho, profundidad?.

## *Kernels*

Una función del tipo **Kernel** es una función de *GPU* que debe llamarse desde el código de la *CPU*. Esta tiene dos características fundamentales:

1.   Las funciones de tipo **Kernel** no pueden devolver explícitamente un valor; todos los datos de resultado deben escribirse en una matriz que se pasa a la función (si calcula un escalar, probablemente pasará una matriz de un elemento).
2.   En una función de tipo **Kernel** se declara explícitamente su jerarquía de hilos cuando se les llama: es decir, el número de bloques de hilos y el número de hilos por bloque (hay que tener en cuenta que, si bien un *Kernel* se compila una vez, se puede llamar varias veces con diferentes tamaños de bloque o tamaños de grid).

Nota (1): Los dispositivos *CUDA* más nuevos admiten el lanzamiento del *Kernel* del lado del dispositivo; esta característica se llama paralelismo dinámico.

A primera vista, escribir un función tipo *Kernel* *CUDA* con *Numba* se parece mucho a escribir una función *JIT* para la *CPU*:

El decorador `@cuda.jit`, le indica al interprete de *Python* que esta función debe **ejecutarse en la GPU** y por lo tanto su ejecución sera mucho más rapida.

# Ejemplos

En esta sección vamos a ver algunos ejemplos de *Numba* accediendo a prestaciones de *CUDA*.

## Suma de vectores (*CPU v.s. GPU*)

Aunque la suma de vectores es un algoritmo sencillo de comprender y de implementar, es un excelente ejemplo para mostrar varios conceptos de *CUDA* y a su vez de *Numba*.

Veamos la versión secuencial (*CPU*) de la suma de vectores en *Python*.

In [ ]:
import numpy as np

# suma de vectores secuencial
def suma_vec_sec(arrRes, arrA, arrB):
    # suma de cada entrada de ambos vectores
    for pos in range(len(arrRes)):
        arrRes[pos] = arrA[pos] + arrB[pos]

# vectores a sumar
a = np.array([1, 2, 3, 4, 5])
b = np.array([10, 20, 30, 40, 50])
c = np.array([0, 0, 0, 0, 0])

Es importante notar que esta versión secuencial hace uso del "paso por referencia" ya que el resultado de las operaciones se almacenan en uno de los parámetros que se reciben en la función `suma_vec_sec`.

*Python* trabaja mediante *paso por valor* cuando se hace uso de valores primitivos (*int, float, string, etc.*).

In [ ]:
%%time
# lanzamiento del kernel
suma_vec_sec(c, b, a)

print(c)

[11 22 33 44 55]
CPU times: user 321 µs, sys: 79 µs, total: 400 µs
Wall time: 391 µs


Ahora podemos recordar la forma de implementar y ejecutar este algoritmo mediante *CUDA* puro.

Si das *click* en el siguiente [enlace](https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/main/CUDA/02_Algoritmos_CUDA_SCP.ipynb#scrollTo=6HGthAMmxdLf&line) verás el código que se mostró en el tema de *CUDA*.

Puedes notar que son **más o menos 50 líneas de código**, sin contar el proceso de compilación y ejecución.

Veamos ahora la versión en paralelo de este algoritmo, pero haciando uso de *Numba*.

In [ ]:
from numba import cuda
import numpy as np

# suma de vectores con CUDA mediante numba
'''se puede definir el tipo de datos o dejarlo
   a que lo infiera numba. Si se le indica a numba
   el algoritmo se ejecuta un poco mas rapido
   ¡¡¡PRUEBALO!!!'''
#@cuda.jit('(int32[:], int32[:], int32[:])')
@cuda.jit
def suma_vec_par(arrRes, arrA, arrB):
    # identificador del hilo en el bloque
    tx = cuda.threadIdx.x
    # identificador del bloque en el grid
    ty = cuda.blockIdx.x
    # dimension del bloque (cuantos hilos por bloque)
    bw = cuda.blockDim.x
    # POLINOMIO DE DIRECCIONAMIENTO
    pos = tx + ty * bw
    # limite de los vectores
    if pos < arrRes.size:
        arrRes[pos] = arrA[pos] + arrB[pos]

# vectores a sumar
a = np.array([1, 2, 3, 4, 5])
b = np.array([10, 20, 30, 40, 50])
c = np.array([0, 0, 0, 0, 0])

In [ ]:
%%time
# parametros para lanzar el kernel
hilos_por_bloque = len(a)+1//2
bloques_por_grid = 2

# lanzamiento del kernel
suma_vec_par[hilos_por_bloque, bloques_por_grid](c, b, a)

print(c)

[11 22 33 44 55]
CPU times: user 162 ms, sys: 6.91 ms, total: 169 ms
Wall time: 172 ms


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Lanzamiento de *Kernels*

Cuando se ejecuta (o lanza) una función de tipo *Kernel*, se tiene que indicar el número de [hilos](https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/main/CUDA/01_CUDA_SCP.ipynb#scrollTo=c9kPwGmoNPbY&line) (procesos ligeros a nivel *GPU*) que se van a encargar de ejecutar este *Kernel* en paralelo.

Existen varias formas de lanzar un *Kernel* en *Numba*.

En la celda superior se hace uso de una de las multiples formas de invocar una función de tipo *Kernel*.

Es importante notar que una vez definido el *Kernel* este **no se modifica aunque se invoque con diferente número de hilos**. Ahora veamos otras formas de invocar a este *Kernel*.

Forma 2 de lanzar el *Kernel*

In [ ]:
%%time
# parametros para lanzar el kernel
# dimension del grid (matriz)
griddim = 1, 2
# tamano del bloque de ejecucion
blockdim = 1, 3

# lanzamiento del kernel
suma_vec_par[griddim, blockdim](c, b, a)

print(c)

[11 22 33 44 55]
CPU times: user 3.76 ms, sys: 0 ns, total: 3.76 ms
Wall time: 3.78 ms


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


`griddim` es el número de bloques de hilos por grid. Puede ser:

*   Un entero.
*   Una 1-tupla de enteros.
*   Una 2-tupla de enteros.

`blockdim` es el número de hilos por bloque. Puede ser:

*   Un entero.
*   Una 1-tupla de enteros.
*   Una 2-tupla de enteros.
*   Una 3-tupla de enteros.


Forma 3 de lanzar un *Kernel*

In [ ]:
%%time
# parametros para lanzar el kernel
hilos_por_bloque = 3
blocks_por_grid = (c.size + (hilos_por_bloque - 1)) // hilos_por_bloque

# lanzamiento del kernel
suma_vec_par[blocks_por_grid, hilos_por_bloque](c, b, a)

print(c)

[11 22 33 44 55]
CPU times: user 3.05 ms, sys: 0 ns, total: 3.05 ms
Wall time: 3.47 ms


Hay que notar dos cosas:

*   Cree una instancia del *Kernel* propiamente dicho, especificando un número de bloques (o "bloques por grid") y un número de hilos por bloque. El producto de los dos dará el número total de hilos lanzados. La instanciación del *Kernel* se realiza tomando la función del *Kernel* compilada (aquí `suma_vec_par`) e indexándola con una tupla de enteros.
*   Ejecutando el *Kernel*, pasándole la matriz de entrada (y cualquier matriz de salida separada si es necesario). Por defecto, ejecutar un *Kernel* es sincrónico: la función regresa cuando el *Kernel* ha terminado de ejecutarse y los datos se vuelven a sincronizar.



## Argumentos y funciones de tipo *device*

Recordemos 2 puntos importantes de *CUDA* que se reflejan en *Numba*.

1.   Una funcion de tipo *Kernel* (se ejecuta en el *GPU* pero se manda a llamar desde la *CPU*) no devuelve un valor, ya que los resultados se almacenan en alguno de los parámetros (paso por referencia), tal como se muestra en la siguiente celda.



In [ ]:
# firma del decorador que indica tipo de los argumentos y valor de retorno
@cuda.jit('void(int32[:], int32[:])')
def foo(aryA, aryB):
    "Se procesa la informacion"

2.   Una función de tipo *device* se manda llamar y se ejecuta directamente en la *GPU*, este tipo de funciones pueden devolver valores dentro de la *GPU*, tal como se muestra en la celda inferior.

In [ ]:
'''firma del decorador que indica el tipo de los argumentos y el
   valor de retorno, asi como el tipo de funcion (device)
'''
@cuda.jit('int32(int32, int32)', device=True)
def bar(a, b):
    "Se realiza la suma de los vectores"

En la función anterior, le decimos al programa que ingresamos dos datos de tipo entero (`int32 , int32`) y extraemos otro dato de tipo entero (`int32`).

##Tamaño del bloque

Puede parecer curioso tener una jerarquía de dos niveles al declarar el número de hilos que necesita un *Kernel*. El tamaño del bloque (es decir, el número de subprocesos por bloque) suele ser crucial:

*   En el lado del software, el tamaño del bloque determina cuántos subprocesos comparten un área determinada de memoria compartida.
*   En el lado del hardware, el tamaño del bloque debe ser lo suficientemente grande para la ocupación completa de las unidades de ejecución; Las recomendaciones se pueden encontrar en la Guía de programación *CUDA C/C++*.

Para ayudar a lidiar con matrices multidimensionales, *CUDA* le permite especificar bloques y cuadrículas multidimensionales.

## Posición del hilo (subproceso)

Cuando se ejecuta un *Kernel*, el código de la función del *Kernel* es ejecutado por cada hilo una vez. Por lo tanto, tiene que saber en qué hilo se encuentra, para saber de qué elemento (s) de la matriz es responsable (los algoritmos complejos pueden definir responsabilidades más complejas, pero el principio subyacente es el mismo).

Una forma es que el hilo determine su posición en la grid y en el bloque y calcule manualmente la posición de la matriz correspondiente:

Para un 1D grid:

In [ ]:
tx = cuda.threadIdx.x
bx = cuda.blockIdx.x
bw = cuda.blockDim.x
i = tx + bx * bw
array[i] = something(i)

TypeError: ignored

Para un 2D grid:

In [ ]:
tx = cuda.threadIdx.x
ty = cuda.threadIdx.y
bx = cuda.blockIdx.x
by = cuda.blockIdx.y
bw = cuda.blockDim.x
bh = cuda.blockDim.y
x = tx + bx * bw
y = ty + by * bh
array[x, y] = something(x, y)

TypeError: ignored


`threadIdx`, `blockIdx`, `blockDim` y `gridDim` son objetos especiales proporcionados por el *backend CUDA* con el único propósito de conocer la geometría de la jerarquía de hilos y la posición del hilo actual dentro de esa geometría.

Estos objetos pueden ser 1D, 2D o 3D, dependiendo de cómo se invocó el *Kernel*. Para acceder al valor en cada dimensión, use los atributos $x, y$ y $z$ de estos objetos, respectivamente.

### `numba.cuda.threadIdx`

Indíca el indice del hilo en el bloque actual. Para bloques 1D, el índice (dado por el atributo x) es un número entero que abarca el rango de 0 inclusive a `numba.cuda.blockDim` exclusivo. Existe una regla similar para cada dimensión cuando se utiliza más de una dimensión.

### `numba.cuda.blockDim`

La "forma" (dimensión) del bloque de hilos, como se declaró al crear una instancia del *Kernel*. Este valor es el mismo para todos los hilos en un *Kernel* dado, incluso si pertenecen a bloques diferentes (es decir, cada bloque está "lleno").

### `numba.cuda.blockIdx`

Muestra el indice del bloque en la cuadrícula de hilos que lanzaron un *Kernel*. Para una cuadrícula 1D, el índice (dado por el atributo x) es un número entero que abarca el rango de 0 a `numba.cuda.gridDim` exclusivo. Existe una regla similar para cada dimensión cuando se utiliza más de una dimensión.

### `numba.cuda.gridDim`

La forma de la grid de bloques, es decir, el número total de bloques lanzados por esta invocación del *Kernel*, como se declaró al crear una instancia del *Kernel*.

## Posición del hilo (subproceso) de manera compacta

### `numba.cuda.grid(ndim)`

Devuelve la posición absoluta del hilo actual en toda la cuadrícula de `blocks.ndim` debe corresponder al número de dimensiones declaradas al crear una instancia del *Kernel*.

Si `ndim` es 1, se devuelve un solo entero. Si `ndim` es 2 o 3, se devuelve una tupla del número dado de enteros.


### `numba.cuda.gridsize(ndim)`
Devuelve el tamaño absoluto (o forma) en hilos de toda la cuadrícula de `blocks.ndim` tiene el mismo significado que en `grid()` anterior.

Recordando que para un 1D grid se tenia el siguiente código:

In [ ]:
tx = cuda.threadIdx.x
bx = cuda.blockIdx.x
bw = cuda.blockDim.x
i = tx + bx * bw
array[i] = something(i)

Con la notación compacta de `cuda.grid` queda de la siguiente manera.

Para un 1D grid:

In [ ]:
i = cuda.grid(1)
array[i] = something(i)

Recordando que para un 2D-grid se tenia el siguiente código:

In [ ]:
tx = cuda.threadIdx.x
ty = cuda.threadIdx.y
bx = cuda.blockIdx.x
by = cuda.blockIdx.y
bw = cuda.blockDim.x
bh = cuda.blockDim.y
x = tx + bx * bw
y = ty + by * bh
array[x, y] = something(x, y)

Con la notación compacta de `cuda.grid` queda de la siguiente manera.

Para un 2D-grid:

In [ ]:
x, y = cuda.grid(2)
array[x, y] = something(x, y)

Una implementación la incrementar en 1 los elementos de un vector (`arreglo`), con la notación larga, sería la siguiente.

In [ ]:
@cuda.jit
def incrementeo_en_uno(un_vector):
    # identificar del hilo en un bloque
    tx = cuda.threadIdx.x
    # identificador del bloque
    ty = cuda.blockIdx.x
    # ancho del bloque (cuantos hilos por bloque)
    bw = cuda.blockDim.x
    # POLINOMIO DE DIRECCIONAMIENTO
    pos = tx + ty * bw
    # revisamos limites del arreglo (OUT OF BOUNDS ERROR)
    if pos < un_vector.size:
        un_vector[pos] += 1

Con la notación compacta de `cuda.grid` queda de la siguiente manera.

Para un 1D-grid (vector):

In [ ]:
@cuda.jit
def incrementeo_en_uno(un_vector):
    pos = cuda.grid(1)
    if pos < an_array.size:
        un_vector[pos] += 1

# Ejemplos (vectores)

## Código 1

In [ ]:
from numba import cuda
import numpy as np
@cuda.jit
def increment_by_one(an_array):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    if pos < an_array.size:  # Check array boundaries
        an_array[pos] += 1

n = 32
x = np.arange(n).astype(np.float32)
threads_per_block=32
blocks_per_grid=1
print("Antes de lanzar el Kernel")
print(x)
increment_by_one[threads_per_block,blocks_per_grid](x)
print("Despues de lanzar el Kernel")
print(x)

Antes de lanzar el Kernel
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]
Despues de lanzar el Kernel
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Código 2

In [ ]:
from numba import cuda
import numpy as np
@cuda.jit
def increment_by_one(an_array):
  pos=cuda.grid(1)
  if pos < an_array.size:  # Check array boundaries
    an_array[pos] += 1

n = 32
x = np.arange(n).astype(np.float32)
threads_per_block=32
blocks_per_grid=1
print("Antes de lanzar el Kernel")
print(x)
increment_by_one[threads_per_block,blocks_per_grid](x)
print("Despues de lanzar el Kernel")
print(x)

Antes de lanzar el Kernel
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]
Despues de lanzar el Kernel
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Código 3

In [ ]:
from __future__ import division
from numba import cuda
import numpy
import math

# CUDA kernel
@cuda.jit('void(float32[:])')
def my_kernel(io_array):
    pos = cuda.grid(1)
    if pos < io_array.size:
        io_array[pos] *= 2 # do the computation

# Host code
data = numpy.ones(256).astype(np.float32)
threadsperblock = 256
blockspergrid = math.ceil(data.shape[0] / threadsperblock)
my_kernel[blockspergrid, threadsperblock](data)
print(data)

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Código 4

In [ ]:
from numba import cuda
import numpy as np
import math
@cuda.jit
def sum_arrays(x_in,y_in):
  tId=cuda.threadIdx.x
  bId=cuda.blockIdx.x
  DimBloc=cuda.blockDim.x

  pos=tId+bId*DimBloc

  if pos<x_in.size:
    x_in[pos]=x_in[pos]+y_in[pos]

n = 32
x = np.arange(n).astype(np.float32)
y=np.arange(n).astype(np.float32)

threads_per_block=32
blocks_per_grid=math.ceil(x.size/threads_per_block)

print("Antes de lanzar el Kernel")
print(x)

sum_arrays[blocks_per_grid,threads_per_block](x,y)
print("Despues de lanzar el Kernel")
print(x)

Antes de lanzar el Kernel
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]
Despues de lanzar el Kernel
[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34.
 36. 38. 40. 42. 44. 46. 48. 50. 52. 54. 56. 58. 60. 62.]


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Código 5

In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    tidx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    bidx = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_dimx = cuda.blockDim.x  # number of threads per block
    grid_dimx = cuda.gridDim.x    # number of blocks in the grid

    start = tidx + bidx * block_dimx
    stride = block_dimx * grid_dimx

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 30 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


1.94 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    tidx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    bidx = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_dimx = cuda.blockDim.x  # number of threads per block
    grid_dimx = cuda.gridDim.x    # number of blocks in the grid

    start = tidx + bidx * block_dimx
    stride = block_dimx * grid_dimx

    # assuming x and y inputs are same length
    if start<x.size:
        out[start] = x[start] + y[start]

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 30 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


2.13 ms ± 321 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    start = cuda.grid(1)

    # assuming x and y inputs are same length
    if start<x.size:
        out[start] = x[start] + y[start]

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 30 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


1.95 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


# Ejemplos (matrices)

Bastante compacto y reducido en comparación con *CUDA*, ¿no?.

Para un 2D-grid (matriz):

In [ ]:
@cuda.jit
def incremento_uno_matriz(an_array):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
       an_array[x, y] += 1

Necesitamos definir la matriz con la que se va a trabajar.

In [ ]:
import math
import numpy as np

# matriz de 16x16 llena de unos
matriz = np.ones((16, 16))

# mostramos la matriz inicial
print(matriz)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


El lanzamiento de este *Kernel* podría ser de la siguiente forma:

In [ ]:
%time
hilos_por_bloque = (16, 16)
# redondeamos para ajustarnos a los limites
bloques_por_grid_x = math.ceil(matriz.shape[0] / hilos_por_bloque[0])
bloques_por_grid_y = math.ceil(matriz.shape[1] / hilos_por_bloque[1])
bloques_por_grid = (bloques_por_grid_x, bloques_por_grid_y)
incremento_uno_matriz[bloques_por_grid, hilos_por_bloque](matriz)
print(matriz)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
[[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]]


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


Mucho muy pequeño, compacto y comprensible el código escrito con *Numba*, de hecho con una mínima modificación en la definición de `incremento_uno_matriz` puedes contestar a la pregunta de este [notebook](https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/main/CUDA/02_Algoritmos_CUDA_SCP.ipynb#scrollTo=c9kPwGmoNPbY) respecto a las **operaciones con matrices** mediante *CUDA* o en este caso con *Numba*.

## Código 1

In [ ]:
@cuda.jit
def increment_a_2D_array(an_array):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
       an_array[x, y] += 1

n = 1024
x = np.arange(n).astype(np.float32).reshape((32,32))

threads_per_block = 16
blockdim=threads_per_block , threads_per_block

griddim=math.ceil(n/ blockdim[0]), math.ceil(n/ blockdim[1])

increment_a_2D_array[griddim, blockdim](x)
print(x[:10])

[[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
   15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
   29.  30.  31.  32.]
 [ 33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.  45.  46.
   47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.
   61.  62.  63.  64.]
 [ 65.  66.  67.  68.  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.
   79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.
   93.  94.  95.  96.]
 [ 97.  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110.
  111. 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124.
  125. 126. 127. 128.]
 [129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141. 142.
  143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153. 154. 155. 156.
  157. 158. 159. 160.]
 [161. 162. 163. 164. 165. 166. 167. 168. 169. 170. 171. 172. 173. 174.
  175. 176. 177. 178. 179. 180. 181. 182. 183. 184. 185. 186. 187. 188.
  189. 190. 191. 192.

/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Código 2

In [ ]:
@cuda.jit
def increment_a_2D_array(an_array,an_array1):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
       an_array[x, y] = an_array[x, y] + an_array1[x, y]

n = 1024
x = np.arange(n).astype(np.float32).reshape((32,32))
y=np.arange(n).astype(np.float32).reshape((32,32))
threads_per_block = 16
blockdim=threads_per_block , threads_per_block

griddim=math.ceil(n/ blockdim[0]), math.ceil(n/ blockdim[1])
print(griddim)
increment_a_2D_array[griddim, blockdim](x,y)
print(x[:10])

(64, 64)
[[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.
   28.  30.  32.  34.  36.  38.  40.  42.  44.  46.  48.  50.  52.  54.
   56.  58.  60.  62.]
 [ 64.  66.  68.  70.  72.  74.  76.  78.  80.  82.  84.  86.  88.  90.
   92.  94.  96.  98. 100. 102. 104. 106. 108. 110. 112. 114. 116. 118.
  120. 122. 124. 126.]
 [128. 130. 132. 134. 136. 138. 140. 142. 144. 146. 148. 150. 152. 154.
  156. 158. 160. 162. 164. 166. 168. 170. 172. 174. 176. 178. 180. 182.
  184. 186. 188. 190.]
 [192. 194. 196. 198. 200. 202. 204. 206. 208. 210. 212. 214. 216. 218.
  220. 222. 224. 226. 228. 230. 232. 234. 236. 238. 240. 242. 244. 246.
  248. 250. 252. 254.]
 [256. 258. 260. 262. 264. 266. 268. 270. 272. 274. 276. 278. 280. 282.
  284. 286. 288. 290. 292. 294. 296. 298. 300. 302. 304. 306. 308. 310.
  312. 314. 316. 318.]
 [320. 322. 324. 326. 328. 330. 332. 334. 336. 338. 340. 342. 344. 346.
  348. 350. 352. 354. 356. 358. 360. 362. 364. 366. 368. 370. 372. 374.
  376. 378. 

/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


# Glosario

*Host*: En el contexto de *CUDA* el host es el *CPU* del dispositivo de cómputo en el que se ejecuta el algoritmo.

Asíncrono: En computación un evento (proceso) asíncrono es aquel no tiene correspondencia temporal con otro evento.

# Referencias

1.   https://numba.pydata.org/numba-doc/latest/user/5minguide.html
2.   http://numba.pydata.org/numba-doc/latest/user/threading-layer.html
3.   https://numba.pydata.org/numba-doc/dev/user/jit.html
4.   https://thedatafrog.com/en/articles/make-python-fast-numba/
5.   https://people.duke.edu/~ccc14/sta-663/CUDAPython.html
6. Tolga Soyata: GPU Parallel Program Development Using Cuda.
7. https://fisica.cab.cnea.gov.ar/gpgpu/images/clases/clase_1_cuda.pdf
8. Dongarra Foster: Source Book of parallel computing.




